In [1]:
from copypastelrm.utils.json_tools import read_jsonl_to_list, read_json, save_jsonl
from copypastelrm.metrics.HotpotQA import compute_answer_em_hit_f1
from copypastelrm.metrics.utils import extract_answer_and_facts, extract_answer_and_facts_old

In [2]:
pass_path = "key_data/pass_at_k_equal_0_subset/musique-tpr_1.0-tpp_0.95-enable_thinking_False-tips_threshold_32-1766453865pass_at_0_subset_ids.jsonl"

prompt_paths = [
    "results/infer/test/Qwen3-4B-Instruct-2507/resamples_-1/seed_42/tpr_0.0/copypaste-prompt_copypaste-1766487293.json",
    "results/infer/test/Qwen3-4B-Instruct-2507/resamples_-1/seed_42/tpr_0.0/copypaste-prompt_copypaste-1766590282.json",
    "results/infer/test/Qwen3-4B-Instruct-2507/resamples_-1/seed_42/tpr_0.0/copypaste-prompt_deepseek-1766478707.json",
    "results/infer/test/Qwen3-4B-Instruct-2507/resamples_-1/seed_42/tpr_0.0/copypaste-prompt_deepseek-1766583151.json",
    "results/infer/test/Qwen3-4B-Instruct-2507/resamples_-1/seed_42/tpr_0.0/copypaste-prompt_ircot-1766471827.json",
    "results/infer/test/Qwen3-4B-Instruct-2507/resamples_-1/seed_42/tpr_0.0/copypaste-prompt_rag-1766424839.json",
]

In [8]:
correct_ids = set()

for path in prompt_paths:
    data = read_json(path)['data']
    info = read_json(path)['info']
    prompt_type = info['prompt_type']
    for sample_id, item in data.items():
        dataset = item['dataset']
        if 'musique' not in dataset.lower():
            continue

        predict = item['predict']
        if prompt_type in ['copypaste', 'deepseek']:
            predict_answer, _ = extract_answer_and_facts_old(predict)
        else:
            predict_answer, _ = extract_answer_and_facts(predict)
        
        answer = item['answer']

        em, hit, f1 = compute_answer_em_hit_f1(predict_answer, answer)

        if f1 > 0:
            correct_ids.add(sample_id)

len(correct_ids)


1884

In [9]:
pass_data = set(read_jsonl_to_list(pass_path))
len(pass_data)

1125

In [10]:
real_wrong_ids = pass_data - correct_ids
len(real_wrong_ids)

393

In [ ]:
save_jsonl(list(real_wrong_ids), "key_data/hard/musique_hard.jsonl")